# Tools for Car Object Detection - R-CNN Model

<img width="1109" alt="Screen Shot 2023-10-03 at 12 13 27 PM" src="https://github.com/nickthetj/CarObjectDetection/assets/37059423/bf1b8802-b453-44ea-8dde-fe1a0ddcfe1d">
(Source: Zapp2Photo/Shutterstock.com)

## Overview
Object recognition is the linchpin for self-driving cars. It enables self-driving cars to identify and respond to objects like pedestrians and vehicles in real-time, ensuring safer autonomous driving. This notebook will walk through one of three object detection models that can identify and draw bounding-boxes around cars on the road: YOLOv8, R-CNN, and Bounding Box Regression (BB). The YOLOv8 model was implemented with the pre-packaged model from Ultralytics, and R-CNN and BB were built from the ground up. We will walk through differences between each model on a conceptual level, their implementation and respective evaluation scores. 

## Business Problem
These object detection models were developed as a tool for autonomous vehicles and traffic safety cameras to enable the cameras to perceive and understand its environment by identifying and locating objects of interest in real-time. Specifically in identifying and locating where cars are in a picture.

<img width="1109" alt="Screen Shot 2023-10-03 at 12 13 27 PM" src="https://github.com/nickthetj/CarObjectDetection/assets/37059423/bf1b8802-b453-44ea-8dde-fe1a0ddcfe1d">
(Source: Zapp2Photo/Shutterstock.com)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
import argparse
import pickle
import os


In [24]:
path = "data/training_images/"
annot = "data/annotations"

In [4]:
def compute_iou(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the intersection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [5]:
# loop over the output positive and negative directories
for dirPath in (path+'car', path+'nocar'):
	# if the output directory does not exist yet, create it
	if not os.path.exists(dirPath):
		os.makedirs(dirPath)
		
# grab all image paths in the input images directory
imagePaths = list(paths.list_images(path+'training_images'))

# initialize the total number of positive and negative images we have
# saved to disk so far
totalPositive = 0
totalNegative = 0

In [32]:
# loop over the image paths
for (e, i) in enumerate(os.listdir(annot)):
	# show a progress report
	print(f"[INFO] processing image {i}")
	
	imageName = i.split(".")[0]+".jpg"
	img = cv2.imread(os.path.join(path,imageName))
	df = pd.read_csv(os.path.join(annot,i))
	# extract image dimensions
	(h,w) = img.shape[:2]
	gtBoxes = []
	for row in df.iterrows():
		# extract bounding box coordinates
		x1 = int(row[1][0].split(" ")[0])
		y1 = int(row[1][0].split(" ")[1])
		x2 = int(row[1][0].split(" ")[2])
		y2 = int(row[1][0].split(" ")[3])
	
		# truncate any bounding box coordinates that may fall
		# outside the boundaries of the image
		x1 = max(0, x1)
		y1 = max(0, y1)
		x2 = min(w, x2)
		y2 = min(h, y2)
		
		gtBoxes.append((x1, y1, x2, y2))
	
	# run selective search on the image and initialize our list of
	# proposed boxes
	ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
	ss.setBaseImage(img)
	ss.switchToSelectiveSearchFast()
	rects = ss.process()
	proposedRects= []
	# loop over the rectangles generated by selective search
	for (x, y, w, h) in rects:
		# convert our bounding boxes from (x, y, w, h) to (startX,
		# startY, startX, endY)
		proposedRects.append((x, y, x + w, y + h))

	# initialize counters used to count the number of positive and
	# negative ROIs saved thus far
	positiveROIs = 0
	negativeROIs = 0
	# loop over the maximum number of region proposals
	for proposedRect in proposedRects[:2000]:
		# unpack the proposed rectangle bounding box
		(propStartX, propStartY, propEndX, propEndY) = proposedRect
		# loop over the ground-truth bounding boxes
		for gtBox in gtBoxes:
			# compute the intersection over union between the two
			# boxes and unpack the ground-truth bounding box
			iou = compute_iou(gtBox, proposedRect)
			(gtStartX, gtStartY, gtEndX, gtEndY) = gtBox
			# initialize the ROI and output path
			roi = None
			outputPath = None
			# check to see if the IOU is greater than 70% *and* that
			# we have not hit our positive count limit
			if iou > 0.7 and positiveROIs <= 3:
				# extract the ROI and then derive the output path to
				# the positive instance
				roi = img[propStartY:propEndY, propStartX:propEndX]
				filename = "{}.png".format(totalPositive)
				outputPath = os.path.sep.join(['data/car/',
					filename])
				# increment the positive counters
				positiveROIs += 1
				totalPositive += 1
			# determine if the proposed bounding box falls *within*
			# the ground-truth bounding box
			fullOverlap = propStartX >= gtStartX
			fullOverlap = fullOverlap and propStartY >= gtStartY
			fullOverlap = fullOverlap and propEndX <= gtEndX
			fullOverlap = fullOverlap and propEndY <= gtEndY
				# check to see if there is not full overlap *and* the IoU
			# is less than 5% *and* we have not hit our negative
			# count limit
			if not fullOverlap and iou < 0.05 and negativeROIs <= 3:
				# extract the ROI and then derive the output path to
				# the negative instance
				roi = img[propStartY:propEndY, propStartX:propEndX]
				filename = "{}.png".format(totalNegative)
				outputPath = os.path.sep.join(['data/nocar/',
					filename])
				# increment the negative counters
				negativeROIs += 1
				totalNegative += 1
			# check to see if both the ROI and output path are valid
			if roi is not None and outputPath is not None:
				# resize the ROI to the input dimensions of the CNN
				# that we'll be fine-tuning, then write the ROI to
				# disk
				roi = cv2.resize(roi, (224, 224),
					interpolation=cv2.INTER_CUBIC)
				cv2.imwrite(outputPath, roi)
			# print(f'Positive ROIs: {positiveROIs}, Negative ROIs: {negativeROIs}')
			

[INFO] processing image vid_4_30000.csv
[INFO] processing image vid_4_9660.csv
[INFO] processing image vid_4_28840.csv
[INFO] processing image vid_4_17440.csv
[INFO] processing image vid_4_21680.csv
[INFO] processing image vid_4_26460.csv
[INFO] processing image vid_4_12060.csv
[INFO] processing image vid_4_1980.csv
[INFO] processing image vid_4_8740.csv
[INFO] processing image vid_4_29960.csv
[INFO] processing image vid_4_6180.csv
[INFO] processing image vid_4_26500.csv
[INFO] processing image vid_4_19900.csv
[INFO] processing image vid_4_16400.csv
[INFO] processing image vid_4_12100.csv
[INFO] processing image vid_4_18820.csv
[INFO] processing image vid_4_9700.csv
[INFO] processing image vid_4_6340.csv
[INFO] processing image vid_4_10500.csv
[INFO] processing image vid_4_11420.csv
[INFO] processing image vid_4_19040.csv
[INFO] processing image vid_4_21520.csv
[INFO] processing image vid_4_15040.csv
[INFO] processing image vid_4_21440.csv
[INFO] processing image vid_4_17680.csv
[INFO]